In [1]:

# leba3207

import numpy as np
import pandas as pd

In [2]:

data_folder = 'data/'

journal_file = 'api_journal11-13-17.csv'
price_file = 'api_price11-13-17.csv'
influence_file = 'estimated-article-influence-scores-2015.csv'

journal = pd.read_csv(data_folder + journal_file, sep=',', encoding='latin1')
price = pd.read_csv(data_folder + price_file, sep=',')
influence = pd.read_csv(data_folder + influence_file, sep=',')


In [3]:

def get_uniqueness_attributes(table):
    return table.nunique(axis=0)


def get_ratio_missing_values(table):
    return table.isnull().sum() * 100 / len(table)


def lowercase_columns(table, headers):
    for header in headers:
        table[header] = table[header].str.lower()

In [4]:

"""
# Question 1: Exploration-Description
## PrÃ©senter une description de chacun des attributs des 3 tables, avec des graphiques pour la visualisation des 
statistiques descriptives au besoin.

### Table journal
"""

'\n# Question 1: Exploration-Description\n## PrÃ©senter une description de chacun des attributs des 3 tables, avec des graphiques pour la visualisation des \nstatistiques descriptives au besoin.\n\n### Table journal\n'

In [5]:

print(journal.head())

        issn                                       journal_name  \
0  0001-527X                           Acta Biochimica Polonica   
1  0002-0397                                    Africa Spectrum   
2  0003-0090  Bulletin of the American Museum of Natural His...   
3  0003-5521                                    L'anthropologie   
4  0004-1254  Arhiv Za Higijenu Rada I Toksikologiju-Archive...   

                      pub_name  is_hybrid                    category  url  
0     ACTA BIOCHIMICA POLONICA          0  MOLECULAR AND CELL BIOLOGY  NaN  
1                          NaN          0                         NaN  NaN  
2  AMER MUSEUM NATURAL HISTORY          0       ECOLOGY AND EVOLUTION  NaN  
3                     Elsevier          1                Anthropology  NaN  
4                          NaN          0                         NaN  NaN  


In [6]:

"""
issn: identifiant du journal

journal_name: nom textuel du journal

pub_name: nom de l'Ã©diteur du journal

is_hybrid: valeur boolÃ©enne indiquant si le journal possÃ¨de son propre Ã©diteur Ã  son nom (??)

category: renseigne sur les catÃ©gories de sujet abordÃ©s par le journal 
Il peut exister de multiples valeurs pour chaque objet, leur sÃ©paration ne semble d'ailleurs pas consistante.

url: indique l'adresse url du site du journal


Les attributs journal_name, pub_name et category Ã©tant des donnÃ©es textuelles trÃ¨s inconsistantes, je dÃ©cide avant tout
traitement et Ã©tude supplÃ©mentaire de transformer les valeurs en minuscule pour limiter au maximum l'inconsistence.
"""

"\nissn: identifiant du journal\n\njournal_name: nom textuel du journal\n\npub_name: nom de l'Ã©diteur du journal\n\nis_hybrid: valeur boolÃ©enne indiquant si le journal possÃ¨de son propre Ã©diteur Ã\xa0 son nom (??)\n\ncategory: renseigne sur les catÃ©gories de sujet abordÃ©s par le journal \nIl peut exister de multiples valeurs pour chaque objet, leur sÃ©paration ne semble d'ailleurs pas consistante.\n\nurl: indique l'adresse url du site du journal\n\n\nLes attributs journal_name, pub_name et category Ã©tant des donnÃ©es textuelles trÃ¨s inconsistantes, je dÃ©cide avant tout\ntraitement et Ã©tude supplÃ©mentaire de transformer les valeurs en minuscule pour limiter au maximum l'inconsistence.\n"

In [7]:

lowercase_columns(journal, ['journal_name', 'pub_name', 'category'])

In [8]:

print(f'Valeurs uniques des attributs de journal:\n{get_uniqueness_attributes(journal)}')
print(f'Ratio de valeurs vides pour les attributs de journal:\n{get_ratio_missing_values(journal)}')

Valeurs uniques des attributs de journal:
issn            13149
journal_name    11572
pub_name         3490
is_hybrid           2
category          112
url              6327
dtype: int64
Ratio de valeurs vides pour les attributs de journal:
issn             0.000000
journal_name     0.000000
pub_name        11.179557
is_hybrid        0.000000
category        48.148148
url             51.623698
dtype: float64


In [9]:

"""
On remarque qu'il existe uniquement deux valeurs pour l'attribut is_hybrid (soit 1 soit 0).
Les attributs category, url prÃ©sentent un nombre consÃ©quent de valeurs manquantes.
"""

"\nOn remarque qu'il existe uniquement deux valeurs pour l'attribut is_hybrid (soit 1 soit 0).\nLes attributs category, url prÃ©sentent un nombre consÃ©quent de valeurs manquantes.\n"

In [10]:

"""
# Question 2: PrÃ©traitement-ReprÃ©sentation
## A. Effectuer un prÃ©traitement des donnÃ©es pour supprimer les duplications et corriger les incohÃ©rences sâ€™il y en a.
"""

# TODO: separator in journal/category : and . |


'\n# Question 2: PrÃ©traitement-ReprÃ©sentation\n## A. Effectuer un prÃ©traitement des donnÃ©es pour supprimer les duplications et corriger les incohÃ©rences sâ€™il y en a.\n'